## Connect to Azure ML Workspace using the v2 AML SDK
The code snippet below retrieves a reference to your AML workspace - you can interact directly with resources in your workspace via the SDK, similar to how you can use the Studio UI.

<b>Note:</b> If running from an Azure Machine Learning Compute Instance, select the `Python 3.10 - SDK v2` kernel as this has the updated AML v2 SDK pre-installed.

In [ ]:
from azure.ai.ml import MLClient, command
from azure.ai.ml.entities import Environment
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential
import mltable

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(ml_client)

## Define variables
Names of relevant AML resources

In [ ]:
# TO-DO: UPDATE THE VARIABLE VALUES BELOW

experiment_name = "<YOUR-EXPERIMENT-NAME>" 
compute_cluster = "<YOUR-COMPUTE-CLUSTER-NAME>"

## Create and register an environment
Using linked conda yaml file

In [ ]:
# TO-DO: CREATE AN ENVIRONMENT FOR YOUR EXPERIMENT USING THE VALUES BELOW

# Base image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
# Conda file: sklearn_env.yml
# Name: yourinitials-sklearn-env

env = ...?

ml_client.environments.create_or_update(env)

### Select input dataset

In [ ]:
# TO-DO: GET A REFERENCE TO YOUR REGISTERED HOME PRICE DATASET BY NAME AND VERSION
dataset_name = '<YOUR-DATASET-NAME>'
dataset_version = '<YOUR-DATASET-VERSION>'
filedataset_asset = ml_client.data.get(name=dataset_name, version=dataset_version)

### Update command job inputs

In [ ]:
# TO-DO: UPDATE THE JOB INPUTS DICTIONARY BELOW TO INCLUDE THE FOLLOWING VALUES:
# target_column: (type: string, default value: MEDV)
# test_percent: (type: number, default value: 0.15)
# model_name: (type: string, default value: YOURINITIALS-HOME-PRICE-PREDICTION)

job_inputs = {
    'input_data': Input(
            type=AssetTypes.MLTABLE, 
            path=filedataset_asset.id,
            mode=InputOutputModes.DIRECT
    ),
}


### Submit command job as new experiment

In [ ]:
# TO-DO: UPDATE THE PARAMETERS PASSED TO THE COMMAND JOB BELOW

command_job = command(
    code=".",  # local path where the code is stored
    command="python train.py --inputs ${{inputs.input_data}} --target_column ${{inputs.target_column}} --model_name ${{inputs.model_name}} --test_percent ${{inputs.test_percent}}",
    inputs=job_inputs,
    environment=env,
    compute=compute_cluster
)
returned_job = ml_client.jobs.create_or_update(command_job(target_column='', test_percent='', model_name=''), experiment_name=experiment_name)
returned_job